In [ ]:
# Jupyter notebook related
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Built-in modules
import os

# Basics of Python data handling and visualization
import numpy as np
import imageio

# Imports from eo-learn and sentinelhub-py
from eolearn.core import (
    EOExecutor,
    EOTask,
    FeatureType,
    LinearWorkflow,
    LoadTask,
)
from eolearn.coregistration import (
    ECCRegistrationTask,
)
from eolearn.features import (
    SimpleFilterTask
)

## Create Timelapse and Coregistered Timelapse

In [ ]:
location = 'gillam_mb_canada'

home_dir = os.environ['HOME']
data_dir = f'{home_dir}/data/northern-cities'

### Declare predicates

In [ ]:
class MaxCCPredicate:
    """
    Predicate for filtering frames based on cloud coverage.
    """
    def __init__(self, maxcc):
        self.maxcc = maxcc

    def __call__(self, mask):
        height, width, depth = mask.shape
        cc = np.sum(mask) / (height * width)
        return cc <= self.maxcc

### Declare tasks

In [ ]:
class MakeGIFTask(EOTask):
    """
    Custom task to create a GIF from an EOPatch.
    """
    def __init__(self, feature, project_dir, frames_per_sec=5):
        self.feature = next(self._parse_features(feature)())
        self.project_dir = project_dir
        self.frames_per_sec = frames_per_sec

    def execute(self, eopatch, *, filename):
        """
        Generates a GIF animation from an EOPatch.
        """
        a = 0.0
        b = 1.0
        with imageio.get_writer(os.path.join(self.project_dir, filename), mode='I', fps=self.frames_per_sec) as writer:
            for image in eopatch[self.feature]:
                bands = image[..., [2, 1, 0]]
                out = np.zeros_like(bands)
                num_bands = bands.shape[2]
                for i in range(num_bands):
                    band = bands[:, :, i]
                    c = np.percentile(bands[:, :, i], 1)
                    d = np.percentile(bands[:, :, i], 99)
                    band = a + (band - c) * (b - a) / (d - c)
                    band[band < a] = a
                    band[band > b] = b
                    out[:, :, i] = band
                out = out * 255
                writer.append_data(out.astype(np.uint8))
        return eopatch

### Define tasks

In [ ]:
# LOAD CLEAN EOPATCHES
load_clean_scenes = LoadTask(f'{data_dir}/{location}/eopatches_clean/')

# FILTER OUT CLOUDY SCENES
filter_cloudy_scenes = SimpleFilterTask((FeatureType.MASK, 'CLM'),
                                        MaxCCPredicate(maxcc=0.01))

# MAKE GIF OF CLEAN SCENES
make_gif_timelapse = MakeGIFTask((FeatureType.DATA, 'FEATURES'),
                                 './')

# COREGISTER CLEAN SCENES
coregister_scenes = ECCRegistrationTask((FeatureType.DATA, 'FEATURES'),
                                        channel=3)

# MAKE GIF OF COREGISTERED SCENES
make_gif_coregistered = MakeGIFTask((FeatureType.DATA, 'FEATURES'),
                                    './')

### Define workflow

In [ ]:
workflow = LinearWorkflow(load_clean_scenes,
                          filter_cloudy_scenes,
                          make_gif_timelapse,
                          coregister_scenes,
                          make_gif_coregistered)

### Visualize workflow

In [ ]:
workflow.dependency_graph()

### Prepare execution arguments

In [ ]:
execution_args = []

for idx in range(15, 35):
    execution_args.append({
        load_clean_scenes: {'eopatch_folder': f'eopatch-{idx:04d}'},
        make_gif_timelapse: {'filename': f'eopatch-{idx:04d}-timelapse.gif'},
        make_gif_coregistered: {'filename': f'eopatch-{idx:04d}-coregistered.gif'},
    })

len(execution_args)

### Execute workflow

In [ ]:
executor = EOExecutor(workflow, execution_args, save_logs=True)
executor.run(workers=1, multiprocess=True)

executor.make_report()

failed_ids = executor.get_failed_executions()
if failed_ids:
    raise RuntimeError(f'Execution failed EOPatches with IDs:\n{failed_ids}\n'
                       f'For more info check report at {executor.get_report_filename()}')